### Import packages to graph results

In [1]:
from glob import glob
import pandas as pd
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'iframe'
import os
import import_ipynb
from shared_notebook import create_dataframe

importing Jupyter notebook from shared_notebook.ipynb
Stored 'colorPallet' (list)
Stored 'bgColor' (str)
Stored 'baseFigureWidth' (int)
Stored 'baseFigureHeight' (int)


### Set default color palete and size for making plots

In [2]:
%store -r colorPallet
%store -r bgColor
%store -r baseFigureWidth
%store -r baseFigureHeight

### glob all the log files into a list, then create a dataframe with all json data

In [3]:
normal_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/bwa/*normal*")

In [4]:
tumor_df = create_dataframe("../logs/openfaas/kops-multi-tenant-spot-1x-8vcpu-16gb/bwa/*tumor*")

# Make bar plots to profile bwa function data

In [28]:
fig = make_subplots(specs = [[{"secondary_y": False}]])

# Add figure title
fig.update_layout(
    barmode='stack',
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="center",
        x=0.47
    ),
    margin=dict(
        t=0,
        b=1,
        l=1,
        r=1,
        autoexpand=True
    ),
    font=dict(
        size=16
    ),
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    plot_bgcolor=bgColor,
    xaxis_title_text='Payloads',
    yaxis_title_text='CPU Time Percentage'
)

profileData = pd.DataFrame()
profileData['payload'] = ["Normal", "Tumor"]
# Metrics for profiling
metrics = ['SoftIrq', 'Kernel', 'IOWait', 'User', 'Idle']
profileData['cpuTotalTime'] = [0, 0]
for metric in metrics:
    normalMean = normal_df[f'cpu{metric}Delta'].mean()
    tumorMean = tumor_df[f'cpu{metric}Delta'].mean()
    profileData[metric] = [normalMean, tumorMean]
    profileData['cpuTotalTime'] = [
        normalMean + profileData['cpuTotalTime'][0],
        tumorMean + profileData['cpuTotalTime'][1]
    ]

#profileData['runtime'] = [round(normal_df['runtime'].mean() / 1000), round(tumor_df['runtime'].mean() / 1000)]
#profileData['runtimeText'] = [f"Runtime:<br>{value} s" for value in profileData['runtime']]
for i, metric in enumerate(metrics):
    profileData[metric] = [
        profileData[metric][0] * 100 / profileData['cpuTotalTime'][0],
        profileData[metric][1] * 100 / profileData['cpuTotalTime'][1]
    ]
    text = [f'{round(val, 2)}%' for val in profileData[metric]]
    fig.add_trace(go.Bar(x = profileData["payload"],
                         y = profileData[metric],
                         name = f"CPU {metric}", marker_color=colorPallet[i],
                         text = text),
                  secondary_y=False)

fig.show()

# Save graph
if not os.path.exists("graphs"):
    os.mkdir("graphs")
fig.write_image("graphs/bwa_profile_bars.png", width=baseFigureWidth, height=baseFigureHeight)